# Observations

- period and calculation_type features all the same value, can drop
- split time series into batches of shorter sequences
- how to impute intermediate missing values? -> take last? interpolate?
- add year, month, absolute int as features for date

- one hot encoding of categorical features: too many, not important enough... drop? keep as single numerical feature?

- duplicate datapoints, same brand, same date, same values - two different compsets...

# TODO:
- align the data by brand
- are there weeks missing in between?
- normalize values


# Discussion
- should we predict by brand or by legal entity?


- standardization in the end after adding additional metrics

In [89]:
import pandas as pd 

In [132]:
DATA_PATH = "/home/david/code/datathon24-personal/data/skylab_instagram_datathon_dataset.csv"

df = pd.read_csv(DATA_PATH, sep=";")

def prepare_data(df, sequence_length=10, prediction_dist=4):

    df = df.drop(columns=["period", "calculation_type", "compset", "compset_group", "legal_entity_name", "ultimate_parent_legal_entity_name", "primary_exchange_name"])
    df["period_end_date"] = pd.to_datetime(df["period_end_date"])

    df = df.rename(columns={'business_entity_doing_business_as_name': 'brand', 'period_end_date': 'date'})

    df = df[df['brand'] != "All Brands"]

    df = df.groupby(['brand', 'date']).first().reset_index()

    def correct_country_name(name):
        country_map = {
            "Hong Kong": "China",
            "China;Hong Kong": "China",
            ";France": "France",
            ";": None,
            "Belgium;": "Belgium"
        }
        if name in country_map.keys():
            return country_map[name]
        return name

    df['domicile_country_name'] = df['domicile_country_name'].apply(correct_country_name)




    categorical_features = ["domicile_country_name"]
    for feature in categorical_features:
        df = pd.get_dummies(df, columns=[feature], prefix=feature, dummy_na=True)

    # TODO: remove bad data


    # TODO: add additional features


    # TODO: normalize values
    normalize_cols = ["followers", "pictures", "videos", "comments", "likes"]

    # TODO: impute missing values
    df = df.fillna(method='ffill')
    

    # TODO: create sequences

    # TODO: split df into brands
    # sequence per brand
    # label

    label_col = None

    sequences = []
    labels = []

    for brand in df['brand'].unique():
        branddf = df[df['brand'] == brand]

        for i in range(len(branddf) - (sequence_length + prediction_dist)):
            sequence = df.iloc[i:i + sequence_length].values
            sequences.append(sequence)

            sequence_labels = df.iloc[i+prediction_dist:i+sequence_length+prediction_dist][label_col].values
            labels.append(sequence_labels)


    
    


    return sequences, labels

# df = preprocess(df)


In [135]:
df.columns

Index(['period', 'period_end_date', 'compset_group', 'compset',
       'business_entity_doing_business_as_name', 'legal_entity_name',
       'domicile_country_name', 'ultimate_parent_legal_entity_name',
       'primary_exchange_name', 'calculation_type', 'followers', 'pictures',
       'videos', 'comments', 'likes'],
      dtype='object')

,brand,date,followers,pictures,videos,comments,likes,domicile_country_name_Australia,domicile_country_name_Belgium,domicile_country_name_Brazil,...,domicile_country_name_New Zealand,domicile_country_name_Philippines,domicile_country_name_Poland,domicile_country_name_Singapore,domicile_country_name_Spain,domicile_country_name_Sweden,domicile_country_name_Switzerland,domicile_country_name_United Kingdom of Great Britain and Northern Ireland,domicile_country_name_United States of America,domicile_country_name_nan


In [114]:
df.columns

Index(['brand', 'date', 'followers', 'pictures', 'videos', 'comments', 'likes',
       'domicile_country_name_Australia', 'domicile_country_name_Belgium',
       'domicile_country_name_Brazil', 'domicile_country_name_Canada',
       'domicile_country_name_China', 'domicile_country_name_Denmark',
       'domicile_country_name_France', 'domicile_country_name_Germany',
       'domicile_country_name_Italy', 'domicile_country_name_Japan',
       'domicile_country_name_Mexico', 'domicile_country_name_NaN',
       'domicile_country_name_Netherlands',
       'domicile_country_name_New Zealand',
       'domicile_country_name_Philippines', 'domicile_country_name_Poland',
       'domicile_country_name_Singapore', 'domicile_country_name_Spain',
       'domicile_country_name_Sweden', 'domicile_country_name_Switzerland',
       'domicile_country_name_United Kingdom of Great Britain and Northern Ireland',
       'domicile_country_name_United States of America',
       'domicile_country_name_nan'],
  

In [91]:

df

,date,compset_group,compset,brand,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,followers,pictures,videos,comments,likes
0,2019-12-07,Luxury & Premium & Mainstream,Soft Luxury,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,22066189.0,97.0,13.0,16239.0,6112205.0
1,2021-01-30,Luxury & Premium & Mainstream,Luxury & Premium & Mainstream,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,24289963.0,54.0,2.0,13358.0,3189144.0
2,2016-05-14,Luxury & Premium & Mainstream,Soft Luxury,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,7137442.0,42.0,8.0,15894.0,2152955.0
3,2019-04-20,Luxury & Premium & Mainstream,Luxury & Premium & Mainstream,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,18901125.0,73.0,9.0,26762.0,5224916.0
4,2023-03-25,Luxury & Premium & Mainstream,US Softlines Analyst Interest List,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,30251374.0,54.0,10.0,25681.0,5760987.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
704308,2017-03-18,Outdoor Gear,Outdoor Gear,Klean Kanteen,Klean Kanteen Inc,NaN,Klean Kanteen Inc,NaN,NaN,50.0,8.0,1271.0,14818.0
704309,2018-10-27,Outdoor Gear,Outdoor Gear,Klean Kanteen,Klean Kanteen Inc,NaN,Klean Kanteen Inc,NaN,163873.0,125.0,2.0,1764.0,34188.0
704310,2023-08-05,Outdoor Gear,Outdoor Gear,Klean Kanteen,Klean Kanteen Inc,NaN,Klean Kanteen Inc,NaN,319485.0,118.0,35.0,1697.0,6328.0
704311,2019-11-02,Outdoor Gear,Outdoor Gear,Klean Kanteen,Klean Kanteen Inc,NaN,Klean Kanteen Inc,NaN,228699.0,129.0,2.0,1972.0,29174.0


In [92]:
tmp = df.groupby(['brand', 'date']).count()
tmp

compset_group  compset  legal_entity_name  \
brand      date                                                    
24S        2017-05-06              4        4                  4   
           2017-05-13              4        4                  4   
           2017-05-20              4        4                  4   
           2017-05-27              4        4                  4   
           2017-06-03              4        4                  4   
...                              ...      ...                ...   
shopDisney 2023-08-19              1        1                  1   
           2023-08-26              1        1                  1   
           2023-09-02              1        1                  1   
           2023-09-09              1        1                  1   
           2023-09-16              1        1                  1   

                       domicile_country_name  \
brand      date                                
24S        2017-05-06                      4   
           2017-05-13                      4   
           2017-05-20                      4   
           2017-05-27                      4   
           2017-06-03                      4   
...                                      ...   
shopDisney 2023-08-19                      1   
           2023-08-26                      1   
           2023-09-02                      1   
           2023-09-09                      1   
           2023-09-16                      1   

                       ultimate_parent_legal_entity_name  \
brand      date                                            
24S        2017-05-06                                  4   
           2017-05-13                                  4   
           2017-05-20                                  4   
           2017-05-27                                  4   
           2017-06-03                                  4   
...                                                  ...   
shopDisney 2023-08-19                                  1   
           2023-08-26                                  1   
           2023-09-02                                  1   
           2023-09-09                                  1   
           2023-09-16                                  1   

                       primary_exchange_name  followers  pictures  videos  \
brand      date                                                             
24S        2017-05-06                      4          0         0       0   
           2017-05-13                      4          0         4       4   
           2017-05-20                      4          0         4       4   
           2017-05-27                      4          0         4       4   
           2017-06-03                      4          0         4       4   
...                                      ...        ...       ...     ...   
shopDisney 2023-08-19                      1          1         1       1   
           2023-08-26                      1          1         1       1   
           2023-09-02                      1          1         1       1   
           2023-09-09                      1          1         1       1   
           2023-09-16                      1          1         1       1   

                       comments  likes  
brand      date                         
24S        2017-05-06         0      0  
           2017-05-13         4      4  
           2017-05-20         4      4  
           2017-05-27         4      4  
           2017-06-03         4      4  
...                         ...    ...  
shopDisney 2023-08-19         1      1  
           2023-08-26         1      1  
           2023-09-02         1      1  
           2023-09-09         1      1  
           2023-09-16         1      1  

[298495 rows x 11 columns]

,brand,date,compset_group,compset,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,followers,pictures,videos,comments,likes
0,24S,2017-05-06,Apparel Retail,Apparel Retail,LVMH Moet Hennessy Louis Vuitton SE,France,LVMH Moet Hennessy Louis Vuitton SE,Euronext Paris,NaN,NaN,NaN,NaN,NaN
1,24S,2017-05-13,Apparel Retail,US Softlines Analyst Interest List,LVMH Moet Hennessy Louis Vuitton SE,France,LVMH Moet Hennessy Louis Vuitton SE,Euronext Paris,NaN,6.0,3.0,57.0,1765.0
2,24S,2017-05-20,Apparel Retail,Apparel Retail,LVMH Moet Hennessy Louis Vuitton SE,France,LVMH Moet Hennessy Louis Vuitton SE,Euronext Paris,NaN,6.0,3.0,57.0,1765.0
3,24S,2017-05-27,Apparel Retail,Apparel Retail,LVMH Moet Hennessy Louis Vuitton SE,France,LVMH Moet Hennessy Louis Vuitton SE,Euronext Paris,NaN,6.0,3.0,57.0,1765.0
4,24S,2017-06-03,Apparel Retail,Apparel Retail,LVMH Moet Hennessy Louis Vuitton SE,France,LVMH Moet Hennessy Louis Vuitton SE,Euronext Paris,NaN,24.0,3.0,109.0,3922.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298490,shopDisney,2023-08-19,Toys & Collectibles,Toys & Collectibles,Disney,United States of America,Disney,New York Stock Exchange,4221812.0,266.0,52.0,3099.0,266166.0
298491,shopDisney,2023-08-26,Toys & Collectibles,Toys & Collectibles,Disney,United States of America,Disney,New York Stock Exchange,4224473.0,265.0,61.0,3494.0,316126.0
298492,shopDisney,2023-09-02,Toys & Collectibles,Toys & Collectibles,Disney,United States of America,Disney,New York Stock Exchange,4228605.0,268.0,61.0,3920.0,341795.0
298493,shopDisney,2023-09-09,Toys & Collectibles,Toys & Collectibles,Disney,United States of America,Disney,New York Stock Exchange,4233764.0,259.0,56.0,3700.0,335231.0


In [57]:
brands = list(df['brand'].unique())


In [59]:
df = df.sort_values(by=['brand', 'date'])

# Group by brand and compute time difference
df['time_since_last'] = df.groupby('brand')['date'].diff().dt.days

# Fill NaN values with 0 for the first datapoint of each brand
df['time_since_last'] = df['time_since_last'].fillna(0).astype(int)

In [68]:
time_since_last_is_0 = df[df['time_since_last'] == 0].groupby('brand').count()
time_since_last_is_0

,date,compset_group,compset,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,followers,pictures,videos,comments,likes,time_since_last
brand,,,,,,,,,,,,,
24S,1000,1000,1000,1000,1000,1000,1000,939,996,996,996,996,1000
3.1 Phillip Lim,911,911,911,911,0,911,0,874,911,911,911,911,911
3CE,1,1,1,1,1,1,1,0,1,1,1,1,1
A. Lange & Soehne,404,404,404,404,404,404,404,395,400,401,401,401,404
ANIMALE,1,1,1,1,1,1,1,0,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
bareMinerals,1,1,1,1,1,1,1,0,1,1,1,1,1
dd's Discounts,659,659,659,659,659,659,659,648,659,656,659,659,659
e.l.f.,1,1,1,1,1,1,1,0,1,1,1,1,1


In [69]:
time_since_last_is_0[time_since_last_is_0['date'] > 1]

,date,compset_group,compset,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,followers,pictures,videos,comments,likes,time_since_last
brand,,,,,,,,,,,,,
24S,1000,1000,1000,1000,1000,1000,1000,939,996,996,996,996,1000
3.1 Phillip Lim,911,911,911,911,0,911,0,874,911,911,911,911,911
A. Lange & Soehne,404,404,404,404,404,404,404,395,400,401,401,401,404
Abercrombie & Fitch,911,911,911,911,911,911,911,911,911,911,911,911,911
About You,1210,1210,1210,1210,0,1210,0,429,1206,1203,1206,1206,1210
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zales,807,807,807,807,807,807,807,774,802,804,804,804,807
Zara,1366,1366,1366,1366,1366,1366,1366,1366,1366,1347,1366,1366,1366
Zegna,1366,1366,1366,1366,0,1366,0,1366,1366,1366,1366,1366,1366


In [70]:
df[df['brand'] == "Abercrombie & Fitch"]

,date,compset_group,compset,brand,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,followers,pictures,videos,comments,likes,time_since_last
630685,2015-01-03,Apparel Retail,Omnichannel Apparel Retail,Abercrombie & Fitch,Abercrombie & Fitch,United States of America,Abercrombie & Fitch,New York Stock Exchange,1669930.0,5.0,1.0,908.0,43976.0,0
631556,2015-01-03,Apparel Retail,US Softlines Analyst Interest List,Abercrombie & Fitch,Abercrombie & Fitch,United States of America,Abercrombie & Fitch,New York Stock Exchange,1669930.0,5.0,1.0,908.0,43976.0,0
631570,2015-01-03,Apparel Retail,Apparel Retail,Abercrombie & Fitch,Abercrombie & Fitch,United States of America,Abercrombie & Fitch,New York Stock Exchange,1669930.0,5.0,1.0,908.0,43976.0,0
630782,2015-01-10,Apparel Retail,Apparel Retail,Abercrombie & Fitch,Abercrombie & Fitch,United States of America,Abercrombie & Fitch,New York Stock Exchange,1707596.0,5.0,1.0,545.0,45818.0,7
631189,2015-01-10,Apparel Retail,US Softlines Analyst Interest List,Abercrombie & Fitch,Abercrombie & Fitch,United States of America,Abercrombie & Fitch,New York Stock Exchange,1707596.0,5.0,1.0,545.0,45818.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631077,2023-09-09,Apparel Retail,US Softlines Analyst Interest List,Abercrombie & Fitch,Abercrombie & Fitch,United States of America,Abercrombie & Fitch,New York Stock Exchange,5313591.0,35.0,23.0,1165.0,150449.0,0
631262,2023-09-09,Apparel Retail,Omnichannel Apparel Retail,Abercrombie & Fitch,Abercrombie & Fitch,United States of America,Abercrombie & Fitch,New York Stock Exchange,5313591.0,35.0,23.0,1165.0,150449.0,0
631011,2023-09-16,Apparel Retail,Omnichannel Apparel Retail,Abercrombie & Fitch,Abercrombie & Fitch,United States of America,Abercrombie & Fitch,New York Stock Exchange,5320115.0,38.0,18.0,6617.0,183510.0,7
631679,2023-09-16,Apparel Retail,Apparel Retail,Abercrombie & Fitch,Abercrombie & Fitch,United States of America,Abercrombie & Fitch,New York Stock Exchange,5320115.0,38.0,18.0,6617.0,183510.0,0


,brand,date,compset_group,compset,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,followers,pictures,videos,comments,likes,time_since_last
0,24S,2017-05-06,Apparel Retail,Apparel Retail,LVMH Moet Hennessy Louis Vuitton SE,France,LVMH Moet Hennessy Louis Vuitton SE,Euronext Paris,NaN,NaN,NaN,NaN,NaN,0
1,24S,2017-05-13,Apparel Retail,US Softlines Analyst Interest List,LVMH Moet Hennessy Louis Vuitton SE,France,LVMH Moet Hennessy Louis Vuitton SE,Euronext Paris,NaN,6.0,3.0,57.0,1765.0,7
2,24S,2017-05-20,Apparel Retail,Apparel Retail,LVMH Moet Hennessy Louis Vuitton SE,France,LVMH Moet Hennessy Louis Vuitton SE,Euronext Paris,NaN,6.0,3.0,57.0,1765.0,7
3,24S,2017-05-27,Apparel Retail,Apparel Retail,LVMH Moet Hennessy Louis Vuitton SE,France,LVMH Moet Hennessy Louis Vuitton SE,Euronext Paris,NaN,6.0,3.0,57.0,1765.0,7
4,24S,2017-06-03,Apparel Retail,Apparel Retail,LVMH Moet Hennessy Louis Vuitton SE,France,LVMH Moet Hennessy Louis Vuitton SE,Euronext Paris,NaN,24.0,3.0,109.0,3922.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298490,shopDisney,2023-08-19,Toys & Collectibles,Toys & Collectibles,Disney,United States of America,Disney,New York Stock Exchange,4221812.0,266.0,52.0,3099.0,266166.0,7
298491,shopDisney,2023-08-26,Toys & Collectibles,Toys & Collectibles,Disney,United States of America,Disney,New York Stock Exchange,4224473.0,265.0,61.0,3494.0,316126.0,7
298492,shopDisney,2023-09-02,Toys & Collectibles,Toys & Collectibles,Disney,United States of America,Disney,New York Stock Exchange,4228605.0,268.0,61.0,3920.0,341795.0,7
298493,shopDisney,2023-09-09,Toys & Collectibles,Toys & Collectibles,Disney,United States of America,Disney,New York Stock Exchange,4233764.0,259.0,56.0,3700.0,335231.0,7


In [44]:
df.nunique()

period_end_date                              455
compset_group                                 20
compset                                       54
business_entity_doing_business_as_name       706
legal_entity_name                            423
domicile_country_name                         26
ultimate_parent_legal_entity_name            401
primary_exchange_name                         30
followers                                 243231
pictures                                    8190
videos                                      3748
comments                                   52500
likes                                     197943
dtype: int64

In [49]:
for k, v in df.groupby('ultimate_parent_legal_entity_name')['business_entity_doing_business_as_name'].nunique().sort_values(ascending=False).items():
    print(k, v)

LVMH Moet Hennessy Louis Vuitton SE 23
L'Oreal 18
Anheuser-Busch 18
Estee Lauder 15
Richemont 13
Swatch 12
Arezzo Industria e Comercio SA 9
L'Arche Green NV 8
Societe d'Investissements Cime SA 8
Agnaten SE 7
GRUPO DE MODA SOMA SA 7
Pontegadea Inversiones SL 7
Kering 6
Coca-Cola 6
Nestle 5
Shiseido 5
Signet Jewelers Ltd 5
Foot Locker 5
KAO 5
Authentic Brands Group LLC 4
Pepsi 4
Unilever 4
Veste S.A. Estilo 4
Pentland Group Ltd 4
Genesco Inc 4
General Mills 4
Inbrands SA 4
VF Corporation 4
Gap 4
Danone 4
Anta International Group Holdings Ltd 4
Mars 4
AbbVie 3
Diageo 3
Darden Restaurants 3
PVH Corporation 3
Disney 3
Boohoo 3
Boardriders Inc 3
The TJX Companies 3
Unilever NV 3
Restaurant Brands International 3
Kathmandu (NZ) 3
Asos 3
Yum! Brands 3
Macy's 3
Wolverine World Wide Inc 3
Carter's 3
Revolve Group Inc 3
Henkel AG & Co KgaA 3
Ermenegildo Zegna 3
Hillhouse Capital Group 3
Capri Holdings 3
Beijing Ruyi Fashion Investment Holding Co Ltd 3
Crocs 2
DSW 2
J M Smucker Co 2
Simply Good Fo

In [14]:
df_by_brand = df.groupby('business_entity_doing_business_as_name')
df_by_brand

,period,period_end_date,compset_group,compset,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,calculation_type,followers,pictures,videos,comments,likes
business_entity_doing_business_as_name,,,,,,,,,,,,,,
24S,1332,1332,1332,1332,1332,1332,1332,1332,1332,1252,1328,1328,1328,1328
3.1 Phillip Lim,1365,1365,1365,1365,1365,0,1365,0,1365,1311,1365,1365,1365,1365
3CE,455,455,455,455,455,455,455,455,455,324,455,455,455,455
A. Lange & Soehne,806,806,806,806,806,806,806,806,806,790,800,802,802,802
ANIMALE,403,403,403,403,403,403,403,403,403,272,403,401,403,403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bareMinerals,455,455,455,455,455,455,455,455,455,430,455,455,455,455
dd's Discounts,987,987,987,987,987,987,987,987,987,972,987,984,987,987
e.l.f.,455,455,455,455,455,455,455,455,455,453,455,455,455,455


In [23]:
def has_nan(group):
    return group.isnull().all()

has_no_stock = df_by_brand['primary_exchange_name'].apply(has_nan)

brands_with_stock = [k for k, v in has_no_stock.items() if not v]
brands_without_stock = [k for k, v in has_no_stock.items() if v]


In [24]:
len(brands_with_stock)

467

In [25]:
len(brands_without_stock)

239

In [26]:
def preprocess(df):
    df = df.copy()
    categorical_features = df.select_dtypes(include=['object', 'category']).columns

    # One-hot encode categorical features while treating NaN as its own category
    for feature in categorical_features:
        df[feature] = df[feature].fillna('NaN')  # Fill NaN values with a placeholder
        df = pd.get_dummies(df, columns=[feature], prefix=feature, dummy_na=True)

    return df


In [27]:
preprocess(df)

,followers,pictures,videos,comments,likes,period_Weekly,period_nan,period_end_date_2015-01-03,period_end_date_2015-01-10,period_end_date_2015-01-17,...,primary_exchange_name_Shanghai Stock Exchange,primary_exchange_name_Shanghai Stock Exchange;Hong Kong Exchange,primary_exchange_name_Singapore Exchange,primary_exchange_name_Six Swiss Exchange,primary_exchange_name_Tokyo Stock Exchange,primary_exchange_name_Toronto Stock Exchange,primary_exchange_name_Warsaw Stock Exchange/Equities/Main Market,primary_exchange_name_nan,calculation_type_Metric Value,calculation_type_nan
0,22066189.0,97.0,13.0,16239.0,6112205.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,24289963.0,54.0,2.0,13358.0,3189144.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,7137442.0,42.0,8.0,15894.0,2152955.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
3,18901125.0,73.0,9.0,26762.0,5224916.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,30251374.0,54.0,10.0,25681.0,5760987.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704308,NaN,50.0,8.0,1271.0,14818.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
704309,163873.0,125.0,2.0,1764.0,34188.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
704310,319485.0,118.0,35.0,1697.0,6328.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
704311,228699.0,129.0,2.0,1972.0,29174.0,True,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False


Timestamp('2019-12-07 00:00:00')